In [1]:
import pandas as pd
import numpy as np
import country_converter as coco
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'
import seaborn as sns
import os

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
wdir = os.path.join(rootdir, '_2_intermediate', 'data')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [3]:
pop = pd.read_csv(os.path.join(wdir, 'pop_world.csv'))

### number of observations





In [4]:
df = pd.read_csv(os.path.join(wdir, 'religions__mastertab.csv'))
df = df[~pd.isnull(df.major_religion)].sort_values(['iso', 'nr'], ascending=[True, False]).reset_index(drop=True).copy(deep=True)
df = df[['iso',
         'major_religion',
         'nr']].drop_duplicates(subset=['iso', 'major_religion']).reset_index(drop=True)
df.nr.sum()

82037564

# Shares

In [5]:
df = pd.read_csv(os.path.join(wdir, 'religions__mastertab.csv'))
df = df[~pd.isnull(df.major_religion)].sort_values(['iso', 'nr'], ascending=[True, False]).reset_index(drop=True).copy(deep=True)
df = df[['iso',
         'major_religion',
         'shr',
         'shre']].drop_duplicates(subset=['iso', 'major_religion']).reset_index(drop=True)
bal = pd.DataFrame([sorted(list(pd.unique(df.iso))*5), ['Christian', 'Muslim', 'Traditional', 'Other', 'No Religion']*20]).T
bal.columns = ['iso', 'major_religion']
df = pd.merge(df, bal, on=['iso', 'major_religion'], how='outer')
df.loc[pd.isnull(df.shr), 'shr'] = 0
df.loc[pd.isnull(df.shre), 'shre'] = 0
df = df.sort_values(['iso', 'major_religion']).reset_index(drop=True).copy(deep=True)

# Unweighted average

In [6]:
uw = df[['major_religion', 'shr', 'shre']].groupby('major_religion').mean().reset_index(drop=False).copy(deep=True)
uw

,major_religion,shr,shre
0,Christian,0.518639,0.518530
1,Muslim,0.353042,0.353960
2,No Religion,0.042775,0.041810
3,Other,0.031910,0.032027
4,Traditional,0.053634,0.053673


# Weighted average

In [7]:
pop = pop[pop.year==1980].reset_index(drop=True).copy(deep=True)
df = pd.merge(df, pop, on='iso', how='left', indicator=True)
print(df._merge.value_counts())
del df['_merge']
df['pop'] = df['pop'] / df.drop_duplicates(subset='iso').reset_index(drop=True)['pop'].sum()
df['shr'] = df.shr*df['pop']
df['shre'] = df.shre*df['pop']

both          110
left_only       0
right_only      0
Name: _merge, dtype: int64


In [8]:
ww = df[['major_religion', 'shr', 'shre']].groupby('major_religion').sum().reset_index(drop=False).copy(deep=True)

In [9]:
ww

,major_religion,shr,shre
0,Christian,0.502288,0.508416
1,Muslim,0.427310,0.422146
2,No Religion,0.029937,0.028450
3,Other,0.011093,0.010938
4,Traditional,0.029372,0.030050


In [10]:
uw

,major_religion,shr,shre
0,Christian,0.518639,0.518530
1,Muslim,0.353042,0.353960
2,No Religion,0.042775,0.041810
3,Other,0.031910,0.032027
4,Traditional,0.053634,0.053673


In [11]:
ww.iloc[:,1:].sum(axis=0)

shr     1.0
shre    1.0
dtype: float64

In [12]:
uw.iloc[:,1:].sum(axis=0)

shr     1.0
shre    1.0
dtype: float64

In [13]:
palette = sns.color_palette(n_colors=5)

# Plotting

## 1. Unweighted `shr`

In [14]:
f, ax = plt.subplots()
w,l,p = ax.pie(uw.shr,
        explode=None,
        labels=uw.major_religion,
        rotatelabels=False,
        autopct='%1.1f%%',
        pctdistance=0.5,
        labeldistance=1.1,
        colors = palette)
f.savefig('{}/_0a_shr_majrel_uw_shr.pdf'.format(outdir), bbox_inches='tight', dpi=100)
plt.close(f)
plt.show()

## 2. Unweighted `shre`

In [15]:
f, ax = plt.subplots()
w,l,p = ax.pie(uw.shre,
        explode=None,
        labels=uw.major_religion,
        rotatelabels=False,
        autopct='%1.1f%%',
        pctdistance=0.5,
        labeldistance=1.1,
        colors = palette)
f.savefig('{}/_0b_shr_majrel_uw_shre.pdf'.format(outdir), bbox_inches='tight', dpi=100)
plt.close(f)
plt.show()

## 3. Weighted `shr`

In [16]:
f, ax = plt.subplots()
w,l,p = ax.pie(ww.shr,
        explode=None,
        labels=ww.major_religion,
        rotatelabels=False,
        autopct='%1.1f%%',
        pctdistance=0.5,
        labeldistance=1.1,
        colors = palette)
f.savefig('{}/_0c_shr_majrel_ww_shr.pdf'.format(outdir), bbox_inches='tight', dpi=100)
plt.close(f)
plt.show()

## 4. Weighted `shre`

In [17]:
f, ax = plt.subplots()
w,l,p = ax.pie(ww.shre,
        explode=None,
        labels=ww.major_religion,
        rotatelabels=False,
        autopct='%1.1f%%',
        pctdistance=0.5,
        labeldistance=1.1,
        colors = palette)
f.savefig('{}/_0d_shr_majrel_ww_shre.pdf'.format(outdir), bbox_inches='tight', dpi=100)
plt.close(f)
plt.show()